### Engineering

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
int x{y};

In [ ]:
step 1: init epoch, num of processed data
step 2: get unprocessed data
step 3: number of workers
step 4: partrition

In [ ]:
#include <iostream>

In [ ]:
int main() {
    int numbers[] = {1, 2, 3, 4, 5};
    
    for (int i; i <= 3; i++) {
        std::cout << numbers[i];
    }
    
    return 0;
}

In [ ]:
kublet, container runtime

In [2]:
n_cycles = 1_000_000

In [ ]:
start = torch.cuda.Event(enable_timing=True)
torch.cuda._sleep(n_cycles)
end = torch.cuda.Event(enable_timing=True)

torch.cuda.synchronize()
elapsed_time = end.elapsed_time(start)

In [ ]:
cycle_per_ms = n_cycles / elapsed_time

In [ ]:
def cuda_sleep(seconds):
    ms = seconds * 1000
    n_cycles = cycle_per_ms * ms
    torch.cuda._sleep(int(n_cycles))

In [ ]:
j partrition in j device

forward j-1 is executed before j
backward j is executed before j-1

- monitor at run-time
- if there're nodes leaved the cluster, reassign tasks
- if there're new nodes entered the cluster, reconfigure communication ring

In [ ]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
hardshit()
end.record()

elapsed_time = end.elapsed_time(start)

In [ ]:
int* p_x = &y;

In [3]:
world_size = 16

In [4]:
tensor_model_parallel_size = 2
pipeline_model_parallel_size = 4

In [5]:
num_pipeline_model_parallel_groups = 4

In [7]:
for i in range(pipeline_model_parallel_size):
    start_rank = i*num_pipeline_model_parallel_groups
    end_rank = (i+1)*num_pipeline_model_parallel_groups
    
    for j in range(tensor_model_parallel_size):
        ranks = list(range(
            start_rank+j,
            end_rank,
            tensor_model_parallel_size
        ))
        
        print(ranks)

[0, 2]
[1, 3]
[4, 6]
[5, 7]
[8, 10]
[9, 11]
[12, 14]
[13, 15]


In [8]:
from transformer_lens import FactoredMatrix

In [ ]:
W_E = model.W_E
W_U = model.W_U

In [ ]:
W_O = model.W_O[layer_idx, head_idx]
W_V = model.W_V[layer_idx, head_idx]

In [ ]:
OV_circuit = FactoredMatrix(A=W_V, B=W_O)

In [ ]:
W_E @ OV_circuit @ W_E.T

In [ ]:
def print_shape(module, inp):
    print(inp.shape)

In [ ]:
model.blocks[1].register_forward_pre_hook(print_shape)

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
residual_directions = model.tokens_to_residual_directions(answer_tokens)
correct_residual_direction, incorrect_residual_direction = residual_directions.unbind(dim=0)

In [ ]:
logit_difference_direction = correct_residual_direction - incorrect_residual_direction

In [ ]:
accumulated_residual_streams = cache.accumulated_residual(layer=layer_idx, pos_slice=-1)

In [ ]:
scaled_accumulated_residual_streams = model.apply_ln_to_stack(
    accumulated_residual_streams,
    layer=layer_idx,
    pos_slice=-1
)

In [9]:
from einops import einsum

In [ ]:
logit_difference = einsum(
    scaled_accumulated_residual_streams,
    logit_difference_direction,
    ""
)

In [10]:
for i in range(pipeline_model_parallel_size):
    start_rank = i*num_pipeline_model_parallel_groups
    end_rank = (i+1)*num_pipeline_model_parallel_groups
    
    for j in range(tensor_model_parallel_size):
        ranks = list(range(
            start_rank+j,
            end_rank,
            tensor_model_parallel_size
        ))
        
        print(ranks)

[0, 2]
[1, 3]
[4, 6]
[5, 7]
[8, 10]
[9, 11]
[12, 14]
[13, 15]


In [ ]:
embed = model.embed(tokens)
pos_embed = model.pos_embed(tokens)

In [ ]:
residual = embed + pos_embed

In [ ]:
for block in model.blocks:
    residual = block(residual)

In [ ]:
residual = model.final_ln(residual)
logits = model.unembed(residual)

In [11]:
from transformer_lens import FactoredMatrix

In [ ]:
AB = FactoredMatrix(A=A, B=B)

In [ ]:
AB.eigenvalues

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
target_tokens = tokens[1:]

In [ ]:
logits = model(tokens)

In [ ]:
log_probs = F.softmax(logits[:, -1, :])

In [ ]:
-log_probs[target_tokens].mean()

In [ ]:
W_O = model.W_O[layer_idx, head_idx]

step 1: two prompts
step 2: choose a start compnent, end component
step 3: record the interdimate activations of the corrupted prompt
step 4: run the clean prompt, patch the activation of the start component in the corrupted prompt
step 5: record the activation of the end component in step 4
step 6: rerun the clean prompt, this time, patch the activations of the end component from step 5

In [ ]:
W_O = model.W_O[layer_idx, head_idx]
W_V = model.W_V[layer_idx, head_idx]

In [ ]:
W_E = model.W_E
W_U = model.W_U

In [ ]:
full_OV_circuit = W_E @ W_V @ W_O @ W_U

In [ ]:
embedding = nn.Embedding(num_embeddings=100, embedding_dim=5, padding_idx=5)

In [ ]:
logits = final_residual_stream @ unembed

In [ ]:
W_pos = model.W_pos

In [ ]:
W_Q = model.W_Q[layer_idx, head_idx]
W_K = model.W_K[layer_idx, head_idx]

QK_circuit = W_K @ W_Q

In [ ]:
pos_by_pos_scores = W_pos @ QK_circuit @ W_pos.T

In [ ]:
def mask(scores):
    mask = torch.triu(torch.ones_like(attn_scores)).bool()
    neg_inf = torch.tensor(-1e6)
    

In [ ]:
W_pos = model.W_pos
W_O = model.W_O[layer_idx, head_idx]
W_V = model.W_V[layer_idx, head_idx]

OV_circuit = W_V @ W_O
pos_by_pos_scores = W_pos @ OV_circuit @ W_pos.T

In [ ]:
def mask(scores):
    mask = torch.triu(torch.ones_like(scores)).bool()
    neg_inf = torch.tensor(-1e9)
    return torch.where(mask, scores, neg_inf)

In [ ]:
pos_by_pos_pattern = F.softmax(
    mask(pos_by_pos_scores/(d_head**0.5)),
    dim=-1
)